In [136]:
import time
import matplotlib.pyplot as plt
import numpy as np
import keras
from skimage.filters import gabor_kernel
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
if K.backend()=='tensorflow':
    K.set_image_dim_ordering("tf")
from skimage.color import rgb2gray
from scipy import ndimage as ndi
 
# Import Tensorflow with multiprocessing
import tensorflow as tf
import multiprocessing as mp
 
# Loading the CIFAR-10 datasets
from keras.datasets import cifar10
import cv2
from sklearn.model_selection import train_test_split, StratifiedKFold

In [137]:
batch_size = 32
# 32 examples in a mini-batch, smaller batch size means more updates in one epoch
num_classes = 10
epochs = 24
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [138]:
def grayscale(data, dtype='float32'):
    # luma coding weighted average in video systems
    r, g, b = np.asarray(.3, dtype=dtype), np.asarray(.59, dtype=dtype), np.asarray(.11, dtype=dtype)
    rst = r * data[:, :, :, 0] + g * data[:, :, :, 1] + b * data[:, :, :, 2]
    return rst

In [139]:
def add_dimension(data):
    data = np.array([data])
    #re arange the dimension
    print(data.shape)
    data = np.einsum('hijk->ijkh', data)
    return data

In [140]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
train_selected_amount = 50000
test_selected_amount = 10000
num_classes = 10

init_y_train = y_train[:train_selected_amount]
init_y_test = y_test[:test_selected_amount]

x_train = add_dimension(grayscale(x_train[:train_selected_amount]))
x_test = add_dimension(grayscale(x_test[:test_selected_amount]))
y_train = np_utils.to_categorical(init_y_train, num_classes)
y_test = np_utils.to_categorical(init_y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train  /= 255
x_test /= 255

(1, 100, 32, 32)
(1, 100, 32, 32)


(100, 10)


In [142]:
def custom_gabor(shape, dtype=None):
#    orientation_spread = np.linspace(0, 8, 8) / 4. * np.pi
    pi = np.pi
    orientation_spread = np.array([0, pi/4, pi/2, pi*3/4, pi, pi*5/4, pi*3/2, 2*pi])
    scales = np.linspace(2, 3, 6)
    real_kernels = []
#     size, sigma, theta, lambda, gamma aspect ratio
    for orientation in orientation_spread:
        for scale in scales:
            real_kernel = cv2.getGaborKernel((5, 5), 1, orientation, scale, 1, 0)
#             real_kernel = np.delete(np.delete(real_kernel, -1, 0), -1, 1)
            real_kernels.append(real_kernel)
    real_kernels = np.array([real_kernels])
    real_kernels = np.einsum('hijk->jkhi', real_kernels)
    print(real_kernels.shape)

    real_kernels = K.variable(real_kernels)
#     print(real_kernels.shape)
    random = K.random_normal(shape, dtype=dtype)
#     print('here')
#     print(random)
#     print(random.shape)
    return real_kernels

In [143]:
def base_model(shape):
    model = Sequential()
    model.add(Conv2D(48, (3, 3), padding='same',kernel_initializer=custom_gabor, data_format='channels_last', input_shape=shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    # sgd = SGD(lr = 0.1, decay = 1e-6, momentum=0.9, nesterov=True)
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    # Train model
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [122]:
# cnn_n = base_model(x_train.shape[1:])
# cnn_n.summary()

# cnn = cnn_n.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

(3, 3, 1, 48)
here
Tensor("conv2d_44/random_normal:0", shape=(3, 3, 1, 48), dtype=float32)
(3, 3, 1, 48)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 32, 32, 48)        480       
_________________________________________________________________
activation_43 (Activation)   (None, 32, 32, 48)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 30, 30, 32)        13856     
_________________________________________________________________
activation_44 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 15, 15, 32)        0         
_____________________________________

In [123]:
# score = cnn_n.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

Test loss: 2.3519577980041504
Test accuracy: 0.0


### K fold cross validation

In [147]:
k = 5
scores = []
folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(x_train, init_y_train))
for j, (train_idx, val_idx) in enumerate(folds):
    print('fold ', j)
    x_train_cv = x_train[train_idx]
    y_train_cv = y_train[train_idx]
    x_valid_cv = x_train[val_idx]
    y_valid_cv = y_train[val_idx]
    model = base_model(x_train_cv.shape[1:])
    model.fit(x_train_cv, y_train_cv, batch_size=batch_size, epochs=epochs, validation_data=(x_valid_cv, y_valid_cv), shuffle=True)
    score = model.evaluate(x_test, y_test, verbose=0)
    scores.append(score[1] * 100)
print("average accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(scores), np.std(scores)))

/Users/golfo/anaconda/envs/tensorflow36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


fold  0
(3, 3, 1, 48)
here
Tensor("conv2d_128/random_normal:0", shape=(3, 3, 1, 48), dtype=float32)
(3, 3, 1, 48)
Train on 84 samples, validate on 16 samples
Epoch 1/24
84/84 [==============================] - 2s 19ms/step - loss: 3.5721 - acc: 0.0952 - val_loss: 2.2651 - val_acc: 0.1875
Epoch 2/24
84/84 [==============================] - 0s 4ms/step - loss: 2.7517 - acc: 0.1310 - val_loss: 2.2565 - val_acc: 0.1250
Epoch 3/24
84/84 [==============================] - 0s 5ms/step - loss: 2.5371 - acc: 0.2024 - val_loss: 2.2700 - val_acc: 0.0625
Epoch 4/24
84/84 [==============================] - 0s 5ms/step - loss: 2.5705 - acc: 0.1190 - val_loss: 2.2512 - val_acc: 0.1250
Epoch 5/24
84/84 [==============================] - 0s 5ms/step - loss: 2.5603 - acc: 0.1071 - val_loss: 2.2660 - val_acc: 0.1250
Epoch 6/24
84/84 [==============================] - 0s 5ms/step - loss: 2.4210 - acc: 0.1310 - val_loss: 2.2702 - val_acc: 0.1875
Epoch 7/24
84/84 [==============================] - 0s 5ms/st

Epoch 24/24
89/89 [==============================] - 0s 5ms/step - loss: 2.0740 - acc: 0.2360 - val_loss: 2.2903 - val_acc: 0.1818
fold  5
(3, 3, 1, 48)
here
Tensor("conv2d_148/random_normal:0", shape=(3, 3, 1, 48), dtype=float32)
(3, 3, 1, 48)
Train on 89 samples, validate on 11 samples
Epoch 1/24
89/89 [==============================] - 2s 21ms/step - loss: 2.9279 - acc: 0.1011 - val_loss: 2.3471 - val_acc: 0.0000e+00
Epoch 2/24
89/89 [==============================] - 0s 4ms/step - loss: 2.5193 - acc: 0.1798 - val_loss: 2.3128 - val_acc: 0.1818
Epoch 3/24
89/89 [==============================] - 0s 4ms/step - loss: 2.5933 - acc: 0.1236 - val_loss: 2.3051 - val_acc: 0.0909
Epoch 4/24
89/89 [==============================] - 0s 5ms/step - loss: 2.4858 - acc: 0.1348 - val_loss: 2.3031 - val_acc: 0.0909
Epoch 5/24
89/89 [==============================] - 0s 5ms/step - loss: 2.4824 - acc: 0.1236 - val_loss: 2.3089 - val_acc: 0.0909
Epoch 6/24
89/89 [==============================] - 0s 5

Epoch 11/24
94/94 [==============================] - 0s 4ms/step - loss: 2.1986 - acc: 0.2766 - val_loss: 2.1707 - val_acc: 0.1667
Epoch 12/24
94/94 [==============================] - 1s 6ms/step - loss: 2.2297 - acc: 0.2021 - val_loss: 2.1861 - val_acc: 0.1667
Epoch 13/24
94/94 [==============================] - 1s 6ms/step - loss: 2.2822 - acc: 0.1596 - val_loss: 2.1673 - val_acc: 0.3333
Epoch 14/24
94/94 [==============================] - 0s 5ms/step - loss: 2.2804 - acc: 0.1809 - val_loss: 2.1584 - val_acc: 0.3333
Epoch 15/24
94/94 [==============================] - 0s 5ms/step - loss: 2.2488 - acc: 0.2234 - val_loss: 2.1567 - val_acc: 0.3333
Epoch 16/24
94/94 [==============================] - 1s 6ms/step - loss: 2.3007 - acc: 0.1489 - val_loss: 2.1555 - val_acc: 0.3333
Epoch 17/24
94/94 [==============================] - 0s 5ms/step - loss: 2.2026 - acc: 0.1596 - val_loss: 2.1529 - val_acc: 0.1667
Epoch 18/24
94/94 [==============================] - 1s 6ms/step - loss: 2.1980 - a

Epoch 23/24
91/91 [==============================] - 1s 6ms/step - loss: 2.1163 - acc: 0.2088 - val_loss: 2.1971 - val_acc: 0.1111
Epoch 24/24
91/91 [==============================] - 1s 6ms/step - loss: 2.0657 - acc: 0.2747 - val_loss: 2.1902 - val_acc: 0.1111
average accuracy: 12.60% (+/- 3.41%)


In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("models/model5x5.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model5x5.h5")
print("Saved model to disk")